In [ ]:
import json
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras

In [ ]:
DATA_PATH = "/content/drive/MyDrive/MiniProject/data.json"

def load_data(data_path):
    with open(data_path, "r") as fp:
        data = json.load(fp)

    X = np.array(data["mfcc"])
    y = np.array(data["labels"])
    return X, y

In [ ]:
def prepare_datasets(test_size, validation_size):
    X, y = load_data(DATA_PATH)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
    X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=validation_size)

    return X_train, X_validation, X_test, y_train, y_validation, y_test


In [ ]:
def predict(model, X, y):
    X = X[np.newaxis, ...] 

    prediction = model.predict(X)
    print(prediction)
    predicted_index = np.argmax(prediction, axis=1)
    print("Target: {}, Predicted label: {}".format(y, predicted_index))


In [ ]:
def build_model(input_shape):
    model = keras.Sequential()
    #model.add(keras.layers.Bidirectional(keras.layers.LSTM(1000,return_sequences=True, dropout=0.25,recurrent_dropout=0.1),input_shape=input_shape))
    # 2 LSTM layers
    model.add(keras.layers.LSTM(1000, input_shape=input_shape, return_sequences=True))
    model.add(keras.layers.LSTM(64))
    # dense layer
    model.add(keras.layers.Dense(64, activation='relu'))
    model.add(keras.layers.Dropout(0.3))
    #dense layer    
    model.add(keras.layers.Dense(128, activation='relu'))
    model.add(keras.layers.Dropout(0.3))
    #dense layer    
    model.add(keras.layers.Dense(256, activation='relu'))
    model.add(keras.layers.Dropout(0.3))
    #dense layer    
    model.add(keras.layers.Dense(512, activation='relu'))
    model.add(keras.layers.Dropout(0.3))
    #dense layer    
    model.add(keras.layers.Dense(1024, activation='relu'))
    model.add(keras.layers.Dropout(0.3))
    # output layer
    model.add(keras.layers.Dense(7, activation='softmax'))
    return model

In [ ]:
if __name__ == "__main__":

    X_train, X_validation, X_test, y_train, y_validation, y_test = prepare_datasets(0.25, 0.2)
    input_shape = (X_train.shape[1], X_train.shape[2]) # 185, 13
    model = build_model(input_shape)

    optimiser = keras.optimizers.Adam(learning_rate=0.0001)
    model.compile(optimizer=optimiser,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    model_checkpoint_callback = keras.callbacks.ModelCheckpoint("model_lstm.h5",save_best_only=True)
    model.summary()
    history = model.fit(X_train, y_train, validation_data=(X_validation, y_validation), batch_size=32, epochs=75)
    test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
    val_loss,val_acc = model.evaluate(X_validation,y_validation,verbose=2)
    print('\nTest accuracy:', test_acc)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 185, 1000)         4056000   
_________________________________________________________________
lstm_3 (LSTM)                (None, 64)                272640    
_________________________________________________________________
dense_6 (Dense)              (None, 64)                4160      
_________________________________________________________________
dropout_5 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 128)               8320      
_________________________________________________________________
dropout_6 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 256)              

In [ ]:
json_file = open('model_num.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights("model_num.h5")
print("Loaded model from disk")
optimiser = keras.optimizers.Adam(learning_rate=0.0001)
loaded_model.compile(optimizer=optimiser,loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [ ]:
from keras.models import model_from_json
from keras.models import load_model

model_json = model.to_json()


with open("model_num.json", "w") as json_file:
    json_file.write(model_json)


In [ ]:
model_filename = "lstm_raga_classifier.h5"
print("\nSaving model: " + model_filename)
model.save(model_filename)


Saving model: lstm_raga_classifier.h5


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
    X_to_predict = X_test[99]
    y_to_predict = y_test[99]
    print(X_to_predict)
    print(len(X_to_predict))
    # predict sample
    predict(model, X_to_predict, y_to_predict)

[[-2.53606003e+02  6.58374786e+01  2.87235603e+01 ... -9.43208504e+00
  -1.39437447e+01 -1.01516790e+01]
 [-2.10408936e+02  8.54122772e+01  8.69899940e+00 ... -5.79949045e+00
  -6.18959236e+00 -1.30504789e+01]
 [-1.83913269e+02  1.01584900e+02 -2.31153889e+01 ...  3.94765735e-02
  -3.05618584e-01 -1.58605518e+01]
 ...
 [-3.24085938e+02  1.24378281e+02  1.33855457e+01 ... -1.91381035e+01
  -1.65821381e+01 -1.26598272e+01]
 [-3.23613708e+02  1.32637146e+02  1.63405800e+01 ... -1.85993614e+01
  -1.36715775e+01 -3.61418533e+00]
 [-2.83764709e+02  1.64807846e+02  3.68926392e+01 ... -2.33828306e+00
  -6.59531498e+00  4.84206057e+00]]
185
Target: 6, Predicted label: [6]


In [ ]:
with open('./model_num.json', 'r') as f:
  model = model_from_json(f.read())
model.load_weights('./model_num.h5')

In [ ]:
import math
import librosa

SAMPLE_RATE = 22050
DURATION = 30
SAMPLES_PER_TRACK = SAMPLE_RATE * DURATION

def save_mfcc(n_mfccs=13,n_fft=2048,hop_length=512,num_segments=5):
    data = {
        "mfcc":[]
    }
    num_samples_per_segment = int(SAMPLES_PER_TRACK / num_segments)
    expected_num_mfcc_vectors_per_segment = math.ceil(num_samples_per_segment/hop_length)

    file = "/content/drive/MyDrive/JA RE BADRA BAIRII JA RE JA RE -LATA-RAJINDER KRISHAN - MADAN MOHAN (BAHAANA 1960).wav"
    signal, sr = librosa.load(file,sr=SAMPLE_RATE)
    for s in range(num_segments):
        start_sample = num_samples_per_segment * s #s=0 -> 0
        finish_sample = start_sample + num_samples_per_segment

        mfcc = librosa.feature.mfcc(signal[start_sample:finish_sample],
                                    sr=sr,
                                    n_mfcc=n_mfccs,
                                    n_fft=n_fft,
                                    hop_length=hop_length)

        mfcc = mfcc.T
        
        if len(mfcc)==expected_num_mfcc_vectors_per_segment:
            data["mfcc"].append(mfcc)

    return data["mfcc"]

x = save_mfcc(num_segments=1)
for i in range(0,len(x)):
  predict(model,x[i],6)


Target: 6, Predicted label: [1]



Saving model: lstm_raga_classifier_lstm.h5


In [ ]:
!mkdir -p saved_model

In [ ]:
!nvidia-smi

Thu Apr  8 17:14:58 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------